In [1]:
# General libraries
# import jax
import sys


import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm, multivariate_normal
from IPython.display import clear_output

# Own code
sys.path.append("../")
from utils.data_utils import create_data, create_dgp_data, transformation, standardize
from utils.tvp_models import TVPVARModel

# Suppress scientific notation in numpy
np.set_printoptions(suppress=True)

# Set RNG
np.random.seed(12345)

In [2]:
train = 178 # 2005Q1

M = 3
p = 4
T = train - p
k = M*(M*p+1)

print(f'k: {k}')

k: 39


In [3]:
ds = pd.read_csv("../data/fred_qd.csv")

In [4]:
transform = True

gdp = transformation(ds["GDPC1"].iloc[2:].to_numpy(), 5, transform)
cpi = transformation(ds["CPIAUCSL"].iloc[2:].to_numpy(), 6, transform)
fedfund = transformation(ds["FEDFUNDS"].iloc[2:].to_numpy(), 2, transform)
compi = transformation(ds["PPIACO"].iloc[2:].to_numpy(), 6, transform)
borrowings = transformation(ds["TOTRESNS"].iloc[2:].to_numpy(), 6, transform)
sp500 = transformation(ds["S&P 500"].iloc[2:].to_numpy(), 5, transform)
m2 = transformation(ds["M2REAL"].iloc[2:].to_numpy(), 5, transform)

# Start due to transformation

lag = 2

series_total = [gdp[lag:], cpi[lag:], fedfund[lag:]]
# series_total = [gdp[lag:], cpi[lag:], fedfund[lag:], compi[lag:], borrowings[lag:], sp500[lag:], m2[lag:]]

In [5]:
standardized_series = standardize(series_total, train)

### TVP-VAR with Variational Bayes (VB) - M = 7

In [6]:
complete_T = len(gdp[lag:])
X_complete, y_complete = create_data(series_total,
                                     complete_T,
                                     complete_T-p,
                                     M,
                                     p,
                                     k)

In [ ]:
# SVSS
# Optimizing the shrinkage for M = 7 (default values are optimal for M = 3)

tau_0 = np.arange(0.5, 5, 1)
tau_1 = np.arange(100, 300, 50)

train_gridsearch = 98 #1985Q4

parameter_set = []

for t_0 in tau_0:
    for t_1 in tau_1:
        parameter_set.append((t_0, t_1))

msfe_list = []
number_of_possibilities = len(parameter_set)
print(f'Number of possibilities: {number_of_possibilities}')

for idx, p_set in enumerate(parameter_set):
    
    tau_0, tau_1 = p_set
    
    tvp_svss = TVPVARModel(X_complete, y_complete, p, train)
    tvp_svss.initialize_priors(prior='svss',
                               prior_parameters={'tau_0': tau_0,
                                                 'tau_1': tau_1,
                                                 'pi0': 0.5},
                               )
    
    mt1t, St1t = tvp_svss.train(print_status=False)
    msfe_list.append(tvp_svss.insample_msfe())
    
    print(f'Progress: {(idx+1)}/{number_of_possibilities}')
    
# Clear printing output
clear_output()
    
sorted_indices = np.argsort(msfe_list)

for i in range(number_of_possibilities):
    
    sorted_index = sorted_indices[i]
    
    print(f'MSFE: {msfe_list[sorted_index]} | '
          f'tau_0: {parameter_set[sorted_index][0]} & '
          f'tau_1: {parameter_set[sorted_index][1]}')
    
    
optimal_svss = parameter_set[sorted_indices[0]]
# Optimal pararmeters: 'tau_0': 2.5, 'tu_1': 150, 'pi0': 0.5

In [50]:
# Horseshoe
# Optimizing the shrinkage for M = 7 (default values are optimal for M = 3)

a0_horseshoe = np.arange(10, 30, 10)
b0_horseshoe = np.arange(10, 30, 10)

train_gridsearch = 98 # 1985Q1

parameter_set = []

for a0 in a0_horseshoe:
    for b0 in b0_horseshoe:
        parameter_set.append((a0, b0))

msfe_list = []
number_of_possibilities = len(parameter_set)
print(f'Number of possibilities: {number_of_possibilities}')

for idx, p_set in enumerate(parameter_set):
    
    a0, b0 = p_set
    
    tvp_horseshoe = TVPVARModel(X_complete, y_complete, p, train_gridsearch)
    tvp_horseshoe.initialize_priors(prior='horseshoe',
                               prior_parameters={'a0': a0,
                                                 'b0': b0})
    
    mt1t, St1t = tvp_horseshoe.train(print_status=False)
    msfe_list.append(tvp_horseshoe.insample_msfe())
    
    print(f'Progress: {(idx+1)}/{number_of_possibilities}')
    
# Clear printing output
clear_output()
    
sorted_indices = np.argsort(msfe_list)

for i in range(number_of_possibilities):
    
    sorted_index = sorted_indices[i]
    
    print(f'MSFE: {msfe_list[sorted_index]} | '
          f'a0: {parameter_set[sorted_index][0]} & '
          f'b0: {parameter_set[sorted_index][1]}')
    
optimal_horseshoe = parameter_set[sorted_indices[0]]

MSFE: 0.0006180817246110649 | a0: 10 & b0: 10
MSFE: 0.0006650044322116735 | a0: 20 & b0: 10
MSFE: 0.0006650044322116807 | a0: 10 & b0: 20
MSFE: 0.0007100285232404204 | a0: 20 & b0: 20


In [ ]:
# Lasso
# Optimizing the shrinkage for M = 7 (default values are optimal for M = 3)

lambda_parameters = np.arange(200, 1000, 100)

train_gridsearch = 98 #1985Q4

parameter_set = []

for lambda_param in lambda_parameters:
    parameter_set.append((lambda_param))

msfe_list = []
number_of_possibilities = len(parameter_set)
print(f'Number of possibilities: {number_of_possibilities}')

for idx, p_set in enumerate(parameter_set):
    
    lambda_param = p_set
    
    tvp_lasso = TVPVARModel(X_complete, y_complete, p, train_gridsearch)
    tvp_lasso.initialize_priors(prior='lasso',
                               prior_parameters={'lambda_param': lambda_param})
    
    mt1t, St1t = tvp_lasso.train(print_status=False)
    msfe_list.append(tvp_lasso.insample_msfe())
    
    print(f'Progress: {(idx+1)}/{number_of_possibilities}')
    
# Clear printing output
clear_output()
    
sorted_indices = np.argsort(msfe_list)

for i in range(number_of_possibilities):
    
    sorted_index = sorted_indices[i]
    
    print(f'MSFE: {msfe_list[sorted_index]} | '
          f'lambda: {parameter_set[sorted_index]}')
    
optimal_lambda = parameter_set[sorted_indices[0]]

In [ ]:
# SVSS
# Calculate h-step forecasts (h = 1 .. 8) for M = 7

optimal_svss = (4.5, 250)

tvp_svss = TVPVARModel(X_complete, y_complete, p, train)
tvp_svss.initialize_priors(prior='svss',
                           prior_parameters={'tau_0': optimal_svss[0],
                                             'tau_1': optimal_svss[1],
                                             'pi0':0.5})

h_forecast = 8

msfe_svss, alpl_svss = tvp_svss.calculate_metrics(h_forecast)

In [ ]:
# Horseshoe
# Calculate h-step forecasts (h = 1 .. 8) for M = 7

optimal_horseshoe = (10,10)

tvp_horseshoe = TVPVARModel(X_complete, y_complete, p, train)
tvp_horseshoe.initialize_priors(prior='horseshoe',
                            prior_parameters={'a0': optimal_horseshoe[0],
                                              'b0': optimal_horseshoe[1]})

tvp_horseshoe.print_status = True

h_forecast = 8

msfe_horseshoe, alpl_horseshoe = tvp_horseshoe.calculate_metrics(h_forecast)

In [25]:
tvp_lasso = TVPVARModel(X_complete, y_complete, p, train)

In [19]:
tvp_horseshoe.lambda_t_horseshoe

array([0.04168368, 0.04110235, 0.04027113, 0.0392854 , 0.03839065,
       0.03817755, 0.03704872, 0.03661861, 0.0366519 , 0.03716934,
       0.0378728 , 0.03855978, 0.03946374, 0.03932116, 0.03944167,
       0.03959487, 0.0398879 , 0.04016129, 0.04075584, 0.04030379,
       0.04026295, 0.04032479, 0.04082513, 0.04055993, 0.04111584,
       0.04045756, 0.04012435, 0.03932479, 0.03820151, 0.03748034,
       0.03724198, 0.03841963, 0.03695012, 0.03645738, 0.03711524,
       0.03861034, 0.03715093, 0.03639413, 0.03637   , 0.03740869,
       0.03903832, 0.03872095, 0.03943244, 0.03727595, 0.0364757 ,
       0.03615887, 0.03614124, 0.03565004, 0.03535333, 0.03606735,
       0.03800296, 0.03850487, 0.04058995, 0.03801093, 0.03720508,
       0.03784443, 0.03767673, 0.03930825, 0.03959862, 0.03892859,
       0.03804532, 0.03747768, 0.03680681, 0.03688439, 0.03700979,
       0.03741589, 0.03781245, 0.03846141, 0.03902957, 0.04049509,
       0.04105636, 0.04191307, 0.04259833, 0.04362579, 0.04299

In [ ]:
# Lasso
# Calculate h-step forecasts (h = 1 .. 8) for M = 7

optimal_lambda = 300

tvp_lasso = TVPVARModel(X_complete, y_complete, p, train)
tvp_lasso.initialize_priors(prior='lasso',
                            prior_parameters={'lambda_param': optimal_lambda})

h_forecast = 8

msfe_lasso, alpl_lasso = tvp_lasso.calculate_metrics(h_forecast)

In [ ]:
import pickle

dump_to_disk = (msfe_horseshoe, alpl_horseshoe, msfe_lasso,
                alpl_lasso, optimal_svss, optimal_horseshoe, optimal_lambda)

with open('dumps/mse_alpl_7_non_standard_horseshoe_lasso.pkl', 'wb') as f:
    pickle.dump(dump_to_disk, f, pickle.HIGHEST_PROTOCOL)

In [10]:
print("hello world")

hello world


In [57]:
np.mean(msfe_horseshoe,1)

array([0.04080503, 0.08028772, 0.09561953, 0.14142871, 0.17950264,
       0.22727599, 0.29079865, 0.35616062])

In [ ]:
import pickle

dump_to_disk = (msfe_horseshoe, alpl_horseshoe, optimal_horseshoe)

with open('dumps/mse_alpl_7_non_standard_horseshoe.pkl', 'wb') as f:
    pickle.dump(dump_to_disk, f, pickle.HIGHEST_PROTOCOL)